## Setup

In [ ]:
from tecton_gen_ai.testing import set_dev_mode, print_md

set_dev_mode()

task = "You are an assistant helping a user find a real estate property based on their preferences"
model = {"model": "openai/gpt-4o", "temperature": 0}

## Ingesting properties into vector db

In [ ]:
PROPERTIES = [
    {
        "id": 1,
        "price": 100000,
        "room_count": 2,
        "name": "Meadowbrook Estates",
        "description": "near school",
        "area": 1000,
    },
    {
        "id": 2,
        "price": 220000,
        "room_count": 3,
        "name": "Pinecrest Villas",
        "description": "very quiet",
        "area": 1500,
    },
    {
        "id": 3,
        "price": 280000,
        "room_count": 2,
        "name": "Oceanview Residences",
        "description": "near shopping mall",
        "area": 2200,
    },
    {
        "id": 4,
        "price": 400000,
        "room_count": 5,
        "name": "Maplewood Manor",
        "description": "very special design",
        "area": 2000,
    },
    {
        "id": 5,
        "price": 500000,
        "room_count": 3,
        "name": "Riverstone Heights",
        "description": "with a beautify garden",
        "area": 1800,
    },
]

In [ ]:
from tecton_gen_ai.api import VectorDB

vdb = VectorDB(
    "lancedb",
    embedding="openai/text-embedding-3-small",
    uri="/tmp/lancedb",
    table_name="properties",
)

vdb.ingest(texts = [p["description"] for p in PROPERTIES], ids = [p["id"] for p in PROPERTIES], metadatas=PROPERTIES)
vdb.search("school", top_k=2)

## Retriever as a tool

In [ ]:
from tecton_gen_ai.api import Agent

@vdb.retriever(top_k=2)
def retrieve(query, filter, result):
    """Get properties based on user preferences provided in the query"""
    return result
    
agent = Agent(
    "agent",
    prompt = task,
    tools = [retrieve],
    llm = model
)

In [ ]:
print_md(agent.invoke("find properties near school"))

# Think Differently

## Rule based filtering + LLM reranking

We just need the agent to remember unstructured data

In [ ]:
def get_candidates(
    min_budget: float=0,
    max_budget: float=0,
    min_area: float=0,
    max_area: float=0,
    min_bedrooms: int=0,
    max_bedrooms: int=0,
    other_preferences: str="",
) -> list[dict]:
    """
    Get properties based on the given budget, bedroom count and area range

    Args:

        min_budget (float): Minimum budget, defaults to 0 (not specified)
        max_budget (float): Maximum budget, defaults to 0 (not specified)
        min_area (float): Minimum area, defaults to 0 (not specified)
        max_area (float): Maximum area, defaults to 0 (not specified)
        min_bedrooms (int): Minimum bedroom count, defaults to 0 (not specified)
        max_bedrooms (int): Maximum bedroom count, defaults to 0 (not specified)
        other_preferences (str): Miscellaneous preferences, defaults to empty string (not specified)

    Returns:

        list[dict]: List of properties that match the given budget and room count range
    """
    filtered_properties = [
        p
        for p in PROPERTIES
        if (min_budget == 0 or p["price"] >= min_budget)
        and (max_budget == 0 or p["price"] <= max_budget)
        and (min_area == 0 or p["area"] >= min_area)
        and (max_area == 0 or p["area"] <= max_area)
        and (min_bedrooms == 0 or p["room_count"] >= min_bedrooms)
        and (max_bedrooms == 0 or p["room_count"] <= max_bedrooms)
    ]
    if not other_preferences:
        return filtered_properties
    
    from langchain_core.vectorstores import InMemoryVectorStore
    from langchain_openai import OpenAIEmbeddings

    vector_store = InMemoryVectorStore(OpenAIEmbeddings())
    vector_store.add_texts(
        [p["description"] for p in filtered_properties],
        metadatas=filtered_properties, 
        ids= [p["id"] for p in filtered_properties]
    )
    res=vector_store.search(other_preferences, search_type="mmr", k=2)
    return [r.metadata for r in res]

In [ ]:
from tecton_gen_ai.api import FeatureServiceConfig

user_preference = FeatureServiceConfig(
    service="user_preference",
    workspace="han-stream-1",
)

agent = Agent(
    name="agent",
    prompt=task,
    tools=[get_candidates, user_preference],
    emitters=[user_preference],
    llm=model,
)

In [ ]:
user_id = "user28"
agent.invoke_tool("han-stream-1_user_preference", kwargs={"user_id": user_id})

In [ ]:
from tecton_gen_ai.testing.interactive import chat

chat(agent, context={"user_id": user_id})
#i can consider 1-3 bedrooms, 1000-3000sqft and 100k-500k budget